### Imports

In [17]:
from probabilityPlus import *
from prettytable import *
from agents import *
from ipythonblocks import *
from utils import *

### Cálculo dos descendentes

In [18]:
def desc_rec(front,exps,rede):
    if not front:
        return exps[1:]
    else:
        first=front[0]
        del front[0]
        filhos=rede.variable_node(first).children
        new = [f for f in filhos if f not in exps]
        front += new
        exps+=new
        return desc_rec(front,exps,rede)

def descendentes(var,rede):
    return desc_rec([var],[var],rede)

### Função cond_indep

In [19]:
def cond_indep(X,Y,E,R):
    if Y in descendentes(X,R) or X in descendentes(Y,R) or Y in E or X in E:
        return False
    return (X,Y,E,R)

### Função cond_indep_explica

In [20]:
def cond_indep_explica(X,Y,E,R):   
    #print("Para que X seja independente de Y dado E:")
    #print("• Y não pode ser descendente de X\n• X não pode ser descendente de Y\n• Nem X nem Y podem estar nem em E\n")
    #print("E =",E)
    #print("X =",X)
    #print("Y =",Y)
    #print("Descendentes de " + X + ":",descendentes(X,burglary))
    #print("Descendentes de " + Y + ":",descendentes(Y,burglary))
    #print("\n")

    explicacao = "Por isso, X e Y não são condicionalmente independentes."
    if Y in descendentes(X,R):
        return (False,'Y é descendente de X. ' + explicacao)
    if X in descendentes(Y,R):
        return (False, 'X é descendente de Y. ' + explicacao)
    if Y in E:
        return (False, 'E contém Y. ' + explicacao)
    if X in E:
        return (False, 'E contém X. ' + explicacao)
    return (True, 'X e Y são condicionalmente independentes.')

### Função todos_cond_indeps

In [21]:
def todos_cond_indeps(rede):
    var=set(rede.variables)
    lista=[]
    for x in var:
        otherX=var-{x}
        for y in otherX:
            i=cond_indep_aux(x,y,rede)
            if i:
                lista.append(i)
    return lista

In [22]:
def cond_indep_aux(x,y,rede):
    lista=[]
    listaAux=[]
    var=set(rede.variables)-{x}-{y}
    for i in range(1,len(var)):
        listaAux=list(combinations(var, i)) 
        lista.append(listaAux)  
    ret=[]
    for e in lista:
        if cond_indep(x,y,e,rede)==True:
            ret.append((x,y,e))
    return ret

(esboços.. possivelmente para apagar)

print('Triplos para JohnCalls:',todos_cond_indeps(burglary))

def as_cond_indep(e,rede):
    out=[]
    Ce={e}
    others=set(rede.variables) - Ce
    #print(rede.variables, "var")
    #print(others, "others")
    #print(Ce, "xx")
    for var in others:
        i=cond_indep({X},{Y}, [var],rede)
        print(i, "aaaaaaa")
        if i!=False:
            out.append(i)
    return out

print('Triplos para JohnCalls:',as_cond_indep('JohnCalls',burglary))

def todos_cond_indeps(rede):
    return reduce(operator.concat,[as_cond_indep(var,rede) for var in rede.variables])
                  

print('Todas Todas:',todos_cond_indeps(burglary),"\n\n")


def todos_cond_indeps(r):
    out=[]
    rVariables=set(r.variables)
    conjuntosE=itertools.combinations(iterable, r.variables)
    print(r.variables)
    for x in rVariables:
        for y in rVariables:
            for e in conjuntosE:
                i=cond_indep({x},{y}, e,r)
                if i==True:
                    out.append({x},{y},e)
    return out
print('Triplos para Burglary:',todos_cond_indeps(burglary))


def as_cond_indep(E,R):
    out=[]
    Ce={E}
    others=set(R.variables) - Ce
    print(R.variables)
    print(others, "others")
    print(Ce, "eee")
    for var in others:
        i=cond_indep(X,var,E,R)
        if i!=False:
            out.append(i)
    return out

R = burglary
print('Triplos para Burglary:',as_cond_indep('JohnCalls',burglary))

## Testes

In [23]:
T, F = True, False
burglary = BayesNet([
    ('Burglary', '', 0.001),
    ('Earthquake', '', 0.002),
    ('Alarm', 'Burglary Earthquake',
     {(T, T): 0.95, (T, F): 0.94, (F, T): 0.29, (F, F): 0.001}),
    ('JohnCalls', 'Alarm', {T: 0.90, F: 0.05}),
    ('MaryCalls', 'Alarm', {T: 0.70, F: 0.01})
    ])

Agora vamos testar para um caso em que Y é descendente de X...

In [24]:
X = 'Burglary'
Y = 'Alarm'
E = 'Earthquake'
print("Descendentes de " + X + ":",descendentes(X,burglary))
print("Descendentes de " + Y + ":",descendentes(Y,burglary))

print(cond_indep_explica(X, Y, E, burglary))

Descendentes de Burglary: ['Alarm', 'JohnCalls', 'MaryCalls']
Descendentes de Alarm: ['JohnCalls', 'MaryCalls']
(False, 'Y é descendente de X. Por isso, X e Y não são condicionalmente independentes.')


Agora vamos testar para um caso em que X é descendente de Y...

In [25]:
X = 'JohnCalls'
Y = 'Alarm'
E = 'Earthquake'
print("Descendentes de " + Y + ":",descendentes(Y,burglary))

print(cond_indep_explica(X, Y, E, burglary))

Descendentes de Alarm: ['JohnCalls', 'MaryCalls']
(False, 'X é descendente de Y. Por isso, X e Y não são condicionalmente independentes.')


### testar os caso dos 2

X é descendente de Y & Y é descendente de X

Agora vamos testar para um caso em que E contém Y...

In [26]:
X = 'Burglary'
Y = 'Earthquake'
E = ['Alarm', 'Earthquake', 'Burglary']

print(cond_indep_explica('Burglary', 'Earthquake', E, burglary))

(False, 'E contém Y. Por isso, X e Y não são condicionalmente independentes.')


Agora vamos testar para um caso em que E contém X...

In [27]:
X = 'Burglary'
Y = 'Earthquake'
E = ['Alarm', 'Burglary']

print(cond_indep_explica('Burglary', 'Earthquake', E, burglary))

(False, 'E contém X. Por isso, X e Y não são condicionalmente independentes.')


In [28]:
print('Triplos para JohnCalls:',todos_cond_indeps(burglary))


Triplos para JohnCalls: []


testar para todos os outros casos....

# IMPORTANTE - a não esquecer

Um triplo (No1, No2, No3) é considerado como sendo do tipo cadeia-causal sempre que, na rede em causa, os 3 nós estão ligados por arcos com a mesma direção.
Isto é, tanto podemos ter No1 -> No2 ->No3 como No1 <- No2 <- No3.


Na verdade, para determinarmos o tipo de um triplo só precisamos de olhar  para a direção dos seus dois arcos.
Por isso só há mesmo 3 tipos de triplos:
- cadeia-causal: arcos na mesma direção 
- causa-comum: arcos em direções opostas, divergindo a partir do nó do meio 
- efeito-comum: arcos em direções opostas, convergindo para o nó do meio

Portanto, qualquer que seja o triplo (No1,No2,No3), o triplo inverso (No3,No2,No1) é sempre do mesmo tipo.

### ---------------------

O conjunto de nós E pode ser representado em Python de diferentes formas:
- uma lista sem elementos repetidos (em que a ordem dos elementos não é relevante)
- um conjunto
- Uma string contendo os nomes das variáveis separadas por espaços

Podem escolher a representação interna que quiserem e desenvolver a função tendo em conta essa representação. 
Idealmente poderão depois estender a função de modo a que aceite esse parâmetro dado sob qualquer dessas formas (tal como, por exemplo em probabilityPlus.py o construtor de um BayesNode permite que os pais de um nó sejam indicados de diferentes formas).

A representação escolhida/permitida para os parâmetros deve ser indicada no relatório.

### ---------------------

Será que tem de ser acrescentar na funcao cond_indep_explica quando Por exemplo:

- X é descendente de Y & Y é descendente de X & E contem X ????

Perguntar aos docentes ????